# Task 4: Conversational Interface - Chat with Llama 3 and Titan Premier LLMs

In this notebook, you build a chatbot using the llama3-8b-instruct and titan-text-premier Foundation Models (FMs) in Amazon Bedrock.

Conversational interfaces such as chatbots and virtual assistants can enhance the user experience for your customers. Chatbots use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. You can use chatbots in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. Users can access them through various channels such as websites, social media platforms, and messaging apps.

- **Chatbot (Basic)**, Zero Shot chatbot with a FM model
- **Chatbot using prompt**, template(LangChain) - Chatbot with some context provided in the prompt template
- **Chatbot with persona**, Chatbot with defined roles. i.e. Career Coach and Human interactions
- **Contextual-aware chatbot**, Passing in context through an external file by generating embeddings.

## LangChain framework for building Chatbot with Amazon Bedrock

In conversational interfaces such as chatbots, remembering previous interactions becomes highly important, both at a short-term and long-term level.

The LangChain framework provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular. Secondly, LangChain provides easy ways to incorporate these utilities into chains, allowing you to easily define and interact with different types of abstractions, which make powerful chatbots easy for you to build.

## Building a Chatbot with Context - Key Elements

The first process in building a context-aware chatbot is to generate embeddings for the context. Typically, you have an ingestion process which runs through your embedding model and generates the embeddings, which will then be stored in a vector store. In this notebook, you use the Titan Embeddings model for this. The second process is user request orchestration, interaction, invoking, and returning the results. This involves orchestrating the user request, interacting with the necessary models/components to gather information, invoking the chatbot to formulate a response, and then returning the chatbot's response back to the user.

## Task 4.1: Environment setup

In this task, you set up your environment.

In [7]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [12]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Task 4.2: Using chat history from LangChain to start the conversation

In this task, you enable the chatbot to carry conversational context across multiple interactions with users. Having a conversational memory is crucial for Chatbots to hold meaningful, coherent dialogues over time.

You implement conversational memory capabilities by building on top of LangChain's InMemoryChatMessageHistory class. This object stores the conversations between the user and the chatbot, and the history is available the chatbot agent so that it can leverage the context from a previous conversation.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The model outputs are non-deterministic.

In [13]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

Human: how are you?
AI: I'm just a language model, I don't have emotions or feelings like humans do. I'm functioning properly and ready to assist with any questions or tasks you may have. How can I help you today?


### New Questions

The model has responded with an initial message. Now, you ask it a few questions.

In [14]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

Starting a new garden can be an exciting and rewarding experience! Here are a few tips to help you get started:

1. **Choose the right location**: Look for a spot that gets at least 6 hours of direct sunlight a day. Make sure the area is level and well-drained. Avoid planting in low-lying areas where water may collect.
2. **Prepare the soil**: Test the pH level of your soil and amend it if necessary. Most vegetables and flowers prefer a slightly acidic to neutral soil pH (between 6.0 and 7.0). Add organic matter like compost or well-rotted manure to improve soil structure and fertility.
3. **Plan your garden**: Decide what you want to grow and how much space you have available. Make a rough sketch of your garden bed and include the mature size of each plant. Consider companion planting, where certain plants are paired together to improve growth or deter pests.
4. **Start small**: Don't try to tackle too much at once. Start with a small garden bed or a few containers and gradually expan

### Build on the questions

Now, ask a question without mentioning the word garden to see if the model can understand the previous conversation.

In [15]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

Bugs! In the context of gardening, bugs can be both beneficial and problematic. Here are some tips on how to deal with common garden bugs:

**Beneficial bugs:**

1. **Ladybugs**: These tiny beetles are natural predators of aphids, whiteflies, and other pests. Encourage ladybugs in your garden by planting flowers that attract them, like marigolds and cosmos.
2. **Lacewings**: These delicate insects are predators of aphids, caterpillars, and other pests. Attract lacewings to your garden by planting flowers like fennel and dill.
3. **Ground beetles**: These nocturnal insects feed on pests like slugs, snails, and caterpillars. Encourage ground beetles by leaving some areas of your garden unmanicured and providing hiding places like rocks and logs.

**Problematic bugs:**

1. **Aphids**: These tiny, sap-sucking insects can spread disease and weaken plants. Control aphids by spraying them off with water, using neem oil, or introducing natural predators like ladybugs.
2. **Whiteflies**: These 

### Finishing this conversation

In [16]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

You're welcome! It was a pleasure chatting with you and helping you with your gardening questions. If you have any more questions or need further assistance in the future, don't hesitate to reach out. Happy gardening!


## Task 4.3: Chatbot using prompt template (LangChain)

In this task, you use the default PromptTemplate that is responsible for the construction of this input. LangChain provides several classes and functions to make constructing and working with prompts easy.

In [17]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Next, start a chat.

In [21]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

Starting chat bot


Output()

In [22]:
print(history)

Human: <|start_header_id|>user<|end_header_id|>
weather of NYC <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI: The weather in New York City (NYC) is known for being quite unpredictable, but I can give you an overview of what to expect throughout the year:

**Spring (March to May)**

* Temperatures gradually warm up, with average highs ranging from 48°F (9°C) in March to 64°F (18°C) in May.
* Expect occasional rain showers and thunderstorms.
* Spring is a great time to visit NYC, with blooming flowers and greenery.

**Summer (June to August)**

* Hot and humid weather, with average highs often reaching 84°F (29°C) or higher.
* July is usually the warmest month, with temperatures sometimes reaching the mid-90s (30s Celsius).
* Summer is peak tourist season, with long days and warm evenings.

**Fall (September to November)**

* Comfortable temperatures, with average highs ranging from 64°F (18°C) in September to 52°F (11°C) in November.
* Expect some rain and occasional coole

## Task 4.4: Chatbot with persona

In this task, Artificial Intelligence(AI) assistant plays the role of a career coach. You can inform the chatbot about its persona (or role) using a system message. Continue to leverage the InMemoryChatMessageHistory class to maintain conversational context.

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

What an exciting field! Artificial Intelligence (AI) has numerous career options across various industries. Here are some of the most in-demand roles:

1. **Machine Learning Engineer**: Design and develop AI models, algorithms, and systems that can learn from data.
2. **Data Scientist**: Analyze and interpret complex data to inform business decisions, often using AI and machine learning techniques.
3. **AI/ML Researcher**: Conduct research and develop new AI and machine learning algorithms, models, and techniques.
4. **Natural Language Processing (NLP) Engineer**: Develop AI-powered language processing systems, such as chatbots, voice assistants, and language translation software.
5. **Computer Vision Engineer**: Design and develop AI-powered computer vision systems, such as facial recognition, object detection, and image classification.
6. **Robotics Engineer**: Develop AI-powered robots that can perform tasks autonomously, such as assembly, logistics, and healthcare.
7. **AI Ethics S

In [26]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

I don't know. I'm a career coach, not a mechanic! If you're having issues with your car, I recommend taking it to a professional mechanic or a trusted repair shop. They'll be able to diagnose and fix the problem.


In [27]:
print(history)

Human: What are the career options in AI?
AI: What an exciting field! Artificial Intelligence (AI) has numerous career options across various industries. Here are some of the most in-demand roles:

1. **Machine Learning Engineer**: Design and develop AI models, algorithms, and systems that can learn from data.
2. **Data Scientist**: Analyze and interpret complex data to inform business decisions, often using AI and machine learning techniques.
3. **AI/ML Researcher**: Conduct research and develop new AI and machine learning algorithms, models, and techniques.
4. **Natural Language Processing (NLP) Engineer**: Develop AI-powered language processing systems, such as chatbots, voice assistants, and language translation software.
5. **Computer Vision Engineer**: Design and develop AI-powered computer vision systems, such as facial recognition, object detection, and image classification.
6. **Robotics Engineer**: Develop AI-powered robots that can perform tasks autonomously, such as assembl

Now, ask a question that is not within this persona's specialty. The model should not answer that question and should give a reason for that.

## Task 4.5 Chatbot with Context

In this task, you ask the chatbot to answer questions based on context that was passed to it. You take a CSV file and use the Titan embeddings model to create a vector representing that context. This vector is stored in Facebook AI Similarity Search (FAISS). When the chatbot is asked a question, you will pass this vector back to the chatbot and have it retrieve the answer using the vector.

### Titan embeddings Model

Embeddings represent words, phrases, or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

You use embeddings for the Retrieval-Augmented Generation (RAG) [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/).

In [28]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS as VectorStore

In order to use embeddings for search, you need a store that can efficiently perform vector similarity searches. In this notebook, you use FAISS, which is an in-memory store. To permanently store vectors, you can use Knowledge Bases for Amazon Bedrock, pgVector, Pinecone, Weaviate, or Chroma.

The LangChain VectorStore APIs are available [here](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [29]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

documents:loaded:size=153
Documents:after split and chunking size=154
vectorstore_faiss_aws:created=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8b18172dd0>::


### Run a quick low code test 

You can use a Wrapper class provided by LangChain to query the vector database store and return the relevant documents. This runs a QA Chain with all default values.

In [30]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

Yes, R is supported with Amazon SageMaker. You can use R within SageMaker notebook instances, which include a preinstalled R kernel and the reticulate library. Reticulate offers an R interface for the Amazon SageMaker Python SDK, enabling ML practitioners to build, train, tune, and deploy R models.


### Chatbot application

For the chatbot, you need context management, history, vector stores, and many other components. You start by building a Retrieval Augmented Generation (RAG) chain that supports context.

This uses the **create_stuff_documents_chain** and **create_retrieval_chain** functions

### Parameters and functions used for RAG

- **Retriever:** You used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types to choose from: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object, where it selects text chunk vectors that are most similar to the question vector.

- **create_stuff_documents_chain** specifies how retrieved context is fed into a prompt and LLM. Retrieved documents are "stuffed" as context without any summarization or other processing into the prompt.

- **create_retrieval_chain** adds the retrieval step and propagates the retrieved context through the chain, providing it alongside the final answer. 

If the question asked falls outside the scope of the context, the model will reply that it doesn't know the answer.

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

{'input': 'What is sagemaker?', 'context': [Document(id='cd62a28d-ce38-46c2-a7f2-b71f6aa1ef34', metadata={'source': '../rag_data/Amazon_SageMaker_FAQs.csv', 'row': 7}, page_content='\ufeffWhat is Amazon SageMaker?: What if I have my own notebook, training, or hosting environment?\nAmazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: Amazon SageMaker provides a full end-to-end workflow, but you can continue to use your existing tools with SageMaker. You can easily transfer the results of each stage in and out of SageMaker as your business requirements dictate.'), Document(id='a0a2de26-0bef-45bc-be48-91835b65a4ba', metadata={'source': '../rag_data/Amazon_SageMaker_FAQs.csv', 'row': 12}, page_content='\ufeffWhat is Amazon SageMaker?: What is Amazon SageMaker Studio?\nAmazon SageMaker is a fully managed service to prepare data and build, train, and dep

Next, start a chat

In [33]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations

Starting chat bot


Output()


You have utilized Titan LLM to create a conversational interface with following patterns:

- Chatbot (Basic - without context)
- Chatbot using prompt template(Langchain)
- Chatbot with personas
- Chatbot with context

### Try it yourself

- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with **Task 5**.